# Introduction to MCP

- The MCP framework connects agents to external capabilities using a client-server protocol.
- Through this protocol, MCP servers can offer,
      tools (which can modify state or connect to external systems), prompts (reusable templates that guide tool usage), and
      resources (like data and content).
- On the client side, agents use LLMs to plan which tool to invoke and format the necessary parameters for the request.
- MCP servers securely manage these requests, acting as an interface to underlying systems (such as databases) and replying to the client in a standardized format.

In [ ]:
!pip install -q  mcp==1.15.0 \
                 langchain==0.3.20 \
                 langchain-mcp-adapters==0.1.11 \
                 langgraph==0.6.9 \
                 langchain_openai==0.3.35 \
                 langgraph==0.6.9 \
                 langchain-community==0.3.19




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 43.7 MB/s eta 0:00:00


## Learning Objective

Implement an MCP-powered Agentic System for generating personalized marketing customer pitches, showcasing the integration of external storage, server-hosted tools with a single agent client.

## Business Case: Marketing Pitch AI Agent
In the marketing industry, crafting personalized customer pitches is essential for engagement and conversion. Teams often spend hours researching customer profiles, tailoring messages, and iterating based on feedback. Traditional methods, rely on manual templates and static tools.

## Problem Scenario

Fragmented research and lack of personalization may result in low response rates, and missed opportunities.  Pitches may overlook key customer details like preferences or recent interactions, causing misalignment with brand voice or campaign goals. This leads to higher churn, wasted ad spend, and suboptimal ROI in competitive markets.

## Proposed Solution

 An automated Marketing Customer Pitch AI Agent powered by an MCP (Model Context Protocol) Agentic System. This system gathers user inputs from external storage (e.g. - User 360°), leverages MCP server-hosted tools for computations (e.g., customer research, pitch scoring), and enables dynamic refinement to generate and adjust tailored pitches.Likely benefits include faster pitch creation, higher personalization accuracy, improved conversion rates, and scalable marketing operations.

In [ ]:
import os
import asyncio
import threading
import time


from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

from mcp.server.fastmcp import FastMCP
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_core.tools import tool
from langchain_mcp_adapters.tools import to_fastmcp
from google.colab import userdata

In [ ]:
openai_api_key = userdata.get('openai_api_key')

### Implementation Plan

The notebook implements an agentic workflow for marketing pitch generation using the langgraph package and MCP for server-hosted tool integration. The execution workflow consists of the following steps:

1. **MCP Server Setup**: Define and expose tools for customer research, pitch scoring, and refinement on the server.
2. **Client Integration**: Load MCP tools into a LangGraph ReAct agent for conversational interaction.
3. **Demonstration**: Run the session to showcase pitch creation & refinment.


## External Database (Simulation)


In [ ]:
# Simulated customer database for research_customer tool
CUSTOMER_DB = {
    "Acme Corp": "Leading manufacturer of road runner catching devices. Known for innovation and quality.",
    "Beta LLC": "Specializes in eco-friendly home products, strong community presence.",
    "Gamma Tech": "Emerging startup in AI-driven logistics software. Focuses on automation and scalability for e-commerce.",
    "Delta Foods": "Family-owned organic food producer emphasizing sustainable farming and direct-to-consumer sales.",
    "Epsilon Media": "Digital marketing agency with expertise in social media campaigns and content creation for B2B clients.",
    "Zeta Renewables": "Provider of solar energy solutions for residential and commercial buildings. Committed to green energy transition."
}

## Server Setup



### Define Tools

Tools for customer info,pitch and scoring are defined here.
_Example Tool Definitions:_
- `research_customer`: This tool will mock data retrieval of customer preferences and history.
- `initial_pitch_prompt` : This tool generates an initial sales pitch prompt based on customer.
- `score_pitch`: Evaluates the effectiveness of a pitch based on predefined criteria.
- `refine_pitch`: Adjusts the pitch based on user feedback to improve its effectiveness and alignment with customer needs.

### Tool: Research Customer
Tool to fetch/mock researched customer data ( from customer 360-degree data storage)


In [ ]:
# Define LangGraph tools using @tool decorators
@tool
def research_customer(name: str) -> str:
    """Lookup customer profile"""
    return CUSTOMER_DB.get(name, "No customer data found.")

### Tool: Score Pitch

Score the genrated pitch on personalization, engagement, and length.



In [ ]:
@tool
def score_pitch(pitch: str) -> str:
    """Use LLM to score pitch"""
    prompt = (
        f"Evaluate the following sales pitch on persuasiveness, clarity, and relevance. "
        f"Score each 1 to 10 and output JSON only:\n{pitch}\n\n"
        "{\"persuasiveness\": int, \"clarity\": int, \"relevance\": int}"
    )
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)  # Use .invoke() for sync call
    import re
    json_match = re.search(r"\{.*\}", response.content, re.DOTALL)
    if json_match:
        return json_match.group(0)
    else:
        # Fallback with error handling
        return '{"persuasiveness": 5, "clarity": 5, "relevance": 5}'

### Tool : Refine Pitch

Refine the pitch based on user feedback. Mock refinement for demo


In [ ]:
@tool
def refine_pitch(pitch: str, feedback: str) -> str:
    """Use LLM to rewrite pitch with feedback"""

    prompt = (
        f"You are an expert sales copywriter specializing in personalized B2B pitches. "
        f"Rewrite the following sales pitch to explicitly incorporate the feedback: make targeted changes "
        f"such as shortening sentences, adding innovative elements, or enhancing calls-to-action. "
        f"Ensure the revised pitch is engaging, concise (under 150 words), and structured as an email "
        f"with a compelling subject line, greeting, body (2-4 paragraphs), and professional sign-off. "
        f"Focus on value, personalization, and the customer's needs to boost persuasiveness.\n\n"
        f"Original Pitch:\n{pitch}\n\n"
        f"Feedback to Incorporate: {feedback}\n\n"
        f"Revised Pitch:"
    )
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)  # Use .invoke() for sync call
    return response.content

### Tool : Initial Pitch

In [ ]:
@tool
def initial_pitch_prompt(customer_name: str, customer_info: str) -> str:
    """Generate an initial sales pitch prompt based on customer name and information."""
    return (
        f"Create a persuasive, personalized sales pitch for {customer_name}. "
        f"Incorporate this customer info: {customer_info}. "
        f"Keep it concise, engaging, and focused on value."
    )

In [ ]:
# Convert LangChain tools to FastMCP
langgraph_tools = [research_customer, score_pitch, refine_pitch, initial_pitch_prompt]
fastmcp_tools = [to_fastmcp(t) for t in langgraph_tools]

In [ ]:
# Create MCP instance for server
mcp = FastMCP("MCP Marketing Pitch Tools", host="127.0.0.1", port=8000, tools=fastmcp_tools)

### Start Server

In [ ]:
# Start the MCP server in a background thread
thread = threading.Thread(
    target=mcp.run,
    kwargs={
        "transport": "streamable-http"
    },
    daemon=True
)
thread.start()

# Brief pause to allow the server to fully start
time.sleep(2)
print("MCP server started on http://127.0.0.1:8000/mcp")

INFO:     Started server process [350]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


MCP server started on http://127.0.0.1:8000/mcp


## Client Setup  

### LLM

In [ ]:
llm = ChatOpenAI(
    api_key=openai_api_key,
    base_url="https://aibe.mygreatlearning.com/openai/v1",
    model='gpt-4o-mini',
    temperature=0
)

###Load Tools and Create Agent

In [ ]:
async def run_pitch_demo(customer_name):
    # Keep session open for entire demo by wrapping everything in async with
    async with streamablehttp_client("http://127.0.0.1:8000/mcp") as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()
            mcp_tools = await load_mcp_tools(session)
            agent = create_react_agent(llm, tools=mcp_tools)


            # Step 1: Use agent to research and generate initial pitch (end-to-end, non-streaming)
            generate_msg = HumanMessage(content=f"Generate a personalized sales pitch for {customer_name}. Research the customer first.")
            try:
                result = await agent.ainvoke({"messages": [generate_msg]})
                # Extract the last AIMessage content from the final state
                messages = result["messages"]
                pitch_msg = next((m for m in reversed(messages) if hasattr(m, 'content') and m.type == 'ai'), None)
                if pitch_msg:
                    pitch = pitch_msg.content
                    print(f"Initial Pitch:\n{pitch.strip()}")
                else:
                    print("No AI response found in agent output.")
                    return  # Early exit if no pitch
            except Exception as e:
                print(f"Error in agent invocation: {e}")
                return

            # Step 2: Score the pitch directly via tool (find by name for robustness, use ainvoke)
            try:
                score_tool = next(t for t in mcp_tools if t.name == "score_pitch")
                score = await score_tool.ainvoke({"pitch": pitch})
                print(f"Pitch score:\n{score}")
            except Exception as e:
                print(f"Error scoring pitch: {e}")

            # Step 3: Refine via tool
            feedback = f"Redraft pitch to improve {score}."
            try:
                refine_tool = next(t for t in mcp_tools if t.name == "refine_pitch")
                refined_pitch = await refine_tool.ainvoke({"pitch": pitch, "feedback": feedback})
                print(f"Refined Pitch:\n{refined_pitch}")
            except Exception as e:
                print(f"Error refining pitch: {e}")
                refined_pitch = None  # Set to None on failure

            # Step 4: Re-score (only if refinement succeeded)
            if refined_pitch:
                try:
                    refined_score = await score_tool.ainvoke({"pitch": refined_pitch})
                    print(f"Refined Pitch score:\n{refined_score}")
                except Exception as e:
                    print(f"Error re-scoring pitch: {e}")
            else:
                print("Skipping re-score due to refinement failure.")



### Execution

In [ ]:
# Execute the demo
# By passing valid customer-name
await run_pitch_demo("Delta Foods")

INFO:     127.0.0.1:51518 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51520 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:51536 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51550 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:51558 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:54088 - "POST /mcp HTTP/1.1" 200 OK
Initial Pitch:
**Sales Pitch for Delta Foods**

---

Hello Delta Foods Team,

At [Your Company Name], we share your passion for sustainability and quality in the food industry. As a family-owned organic food producer, you understand the importance of nurturing both the land and the community. We admire your commitment to sustainable farming practices and your direct-to-consumer sales model, which not only supports local economies but also fosters a deeper connection between consumers and the food they eat.

We believe that our [specific product/service] can enhance your operations by [specific benefit related to sustainability, efficiency, or customer